In [1]:
import numpy as np
import pandas as pd
from keras.utils import image_dataset_from_directory

In [2]:
df="/kaggle/input/hair-type-dataset/data"
train_data=image_dataset_from_directory(
df,
image_size=(256,256),
seed=123,
batch_size=32,
shuffle=True,
validation_split=0.2, 
subset="training"
)
val_data=image_dataset_from_directory(
df,
image_size=(256,256),
seed=123,
batch_size=32,
shuffle=True,
validation_split=0.2, 
subset="validation"
)

Found 1988 files belonging to 5 classes.
Using 1591 files for training.
Found 1988 files belonging to 5 classes.
Using 397 files for validation.


In [3]:
len(train_data)

50

# # QCNN

In [4]:
!pip install cirq 

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 37.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.7/532.7 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.8/202.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 87.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 58.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 67.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 81.8 MB/s e

In [ ]:
import cirq
import tensorflow as tf
from keras.layers import Layer,Dense,Conv2D,MaxPooling2D,Flatten,Input
from keras.models import Model,Sequential
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [ ]:
early_stopping=EarlyStopping(
monitor='val_loss',
patience=5,
verbose=1)

In [ ]:
def build_quantum_circuit(data):
    qubit=[cirq.GridQubit(0,i) for i in range(len(data))]
    circuit=cirq.Circuit()
    for i,value in enumerate(data):
        circuit.append(cirq.rx(data*np.pi).on(qubit[i]))
        circuit.append(cirq.ry(data*np.pi).on(qubit[i]))
        circuit.append(cirq.rz(data*np.pi).on(qubit[i]))
    circuit.append(cirq.CNOT(qubit[0],qubit[1]))
    circuit.append([cirq.measure(qubit[i],key=f"qubit_{i}") for i in range(len(data))])
    return circuit,qubit
def run_quantum_circuit(data):
    results=[]
    for input_data in data:
        circuit,qubit=build_quantum_circuit(input_data)
        simulator=cirq.Simulator()
        result=simulator.run(circuit)
        measure0=result.measurements['qubit_0'].flatten()[0]
        measure1=result.measurements['qubit_1'].flatten()[1]
        combine_result=np.array([measure0,measure1])
        results.append(combine_result)
    return np.array(results,dtype=tf.float32)
class QuantumLayer(Layer):
    def __init__(self):
        super(QuantumLayer,self).__init__()
    def call(self,inputs):
        output=tf.numpy_function(run_quantum_circuit,[inputs],tf.float32)
        output.set_shape((None,2))
#         output = tf.reshape(output, (-1, 1, 2, 1))  # Example: Reshape to (batch_size, 1, 2, 1)

        return output
model=Sequential([
        Input(shape=(256,256,3)),
        Conv2D(32,(3,3),activation='relu',padding="same"),
        Flatten(),

        QuantumLayer(),
        Dense(5,activation="softmax")
    ])
model.compile(optimizer='adam',loss="sparse_categorical_crossentropy",metrics=["accuracy"])
model.summary()

In [ ]:
model.fit(train_data,epochs=50,validation_data=val_data,callbacks=[early_stopping])